In [5]:
import pandas as pd
import numpy as np

DATAFILE = "data/Leoni.dat"
df = pd.read_csv(DATAFILE, sep=";")

y = df.loc[:, "Schlusskurs"].to_numpy()
t = df.loc[:, "Eintrag"].to_numpy()
d = df.loc[:, "Datum"].to_numpy()
u = np.zeros(len(y))

Ts = t[1] - t[2]
# R =
# Q =
df

,Eintrag,Datum,Schlusskurs
0,1,01.06.2006,29.87
1,2,02.06.2006,30.36
2,3,05.06.2006,30.24
3,4,06.06.2006,29.03
4,5,07.06.2006,28.65
...,...,...,...
2995,2996,07.02.2018,58.54
2996,2997,08.02.2018,56.20
2997,2998,09.02.2018,54.94
2998,2999,12.02.2018,54.62
